In [1]:
# python
import sys
import os
import importlib
# columnar analysis
from coffea import processor
import awkward as ak
from dask.distributed import Client
#plotting
import matplotlib.pyplot as plt
# local
sys.path.insert(1, os.path.join(sys.path[0], '../..')) # fixme: there must be a better way to handle this...
from analysis.tools import ffschema, sidm_processor, scaleout
# always reload local modules to pick up changes during development
importlib.reload(ffschema)
importlib.reload(sidm_processor)

<module 'analysis.tools.sidm_processor' from '/home/cms-jovyan/SIDM/analysis/test_notebooks/../../analysis/tools/sidm_processor.py'>

In [2]:
#client = Client("tls://bryan-2ecardwell-40cern-2ech.dask.cmsaf-prod.flatiron.hollandhpc.org:8786")
client = scaleout.make_dask_client("tls://bryan-2ecardwell-40cern-2ech.dask.cmsaf-prod.flatiron.hollandhpc.org:8786")
print(client)

Exception: ModuleNotFoundError("No module named 'analysis'")

In [3]:
fileset = {
    "2Mu2E_100GeV_1p2GeV_9p6mm": [
        "root://xcache//store/group/lpcmetx/SIDM/ffNtupleV4/2018/SIDM_XXTo2ATo2Mu2E_mXX-100_mA-1p2_ctau-9p6_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18DRPremix-102X_upgrade2018_realistic_v15-v1/210326_161703/0000/ffNtuple_1.root",
    ],
}

runner = processor.Runner(
    #executor=processor.IterativeExecutor(),
    #executor=processor.FuturesExecutor(),
    executor=processor.DaskExecutor(client=client),
    schema=ffschema.FFSchema,
)

p = sidm_processor.SidmProcessor(
    [
        "2mu2e",
    ],
    ["base"]
)

# test if processor is serializable
import coffea.util as coffea_util
coffea_util.save(p, "processor.coffea")
print(coffea_util.load("processor.coffea"))

output = runner.run(fileset, treename="ffNtuplizer/ffNtuple", processor_instance=p)

NameError: name 'client' is not defined

In [ ]:
out = output["out"]

In [ ]:
# test that ouput is accumulatable
from coffea.processor import accumulate
double_out = accumulate([output["out"], output["out"]])

In [ ]:
out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table()
out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table(fraction=True)

In [ ]:
double_out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table()
double_out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table(fraction=True)